In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
final = pd.read_csv('국건영_회귀분석용.csv', low_memory = False)

In [3]:
final

,year,ID,sex,age,psu,HE_ht,HE_wt,HE_BMI,HE_sbp_tr,HE_dbp_tr,...,DW_WBT_BMC,DW_SBT_BMD,DW_WBT_BMD,DW_Trk_FT,DW_Trk_MS,DW_SBT_FT,DW_WBT_FT,DX_F_Ts_A,DX_S_Ts_A,DX_FN_Ts_A
0,2008.0,b'A448440901',1.0,70.0,b'A448',155.2,58.0,24.079339,122.3,71.3,...,2113.500475,0.994057,1.119189,5350.559358,28165.253024,9588.845972,10554.355505,-0.893139,-0.537667,-1.857823
1,2008.0,b'A448440902',2.0,68.0,b'A448',165.6,63.4,23.118976,128.7,82.7,...,1616.637082,0.744743,0.851513,11248.239290,30770.948113,21544.371816,22368.168979,-1.420957,-1.534783,-2.336729
2,2008.0,b'A448680102',2.0,48.0,b'A448',152.3,59.3,25.565535,133.7,90.7,...,2274.388678,0.959672,1.237565,12183.372700,28925.836734,22269.339707,23114.327698,1.050435,1.171826,-0.459346
3,2008.0,b'A448680501',2.0,35.0,b'A448',162.5,53.4,20.222485,104.7,70.7,...,2292.951162,1.002821,1.182751,6573.697606,25808.635531,11724.950815,12571.863454,0.961565,0.354087,-0.260561
4,2008.0,b'A448740903',2.0,70.0,b'A448',155.7,63.9,26.358679,111.7,83.7,...,1455.752755,0.772691,0.882857,13489.507467,32492.389825,23471.432798,24379.078925,-1.764957,-3.223913,-2.883364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19496,2011.0,b'O359840901',2.0,60.0,b'O359',154.9,52.2,21.755430,NaN,NaN,...,1602.291088,0.785240,0.902542,9391.419342,25933.167250,16812.840990,17517.341590,-0.790261,-1.645652,-2.023738
19497,2011.0,b'O359859801',1.0,68.0,b'O359',165.2,68.2,24.989887,NaN,NaN,...,2062.206293,0.872690,0.953034,7434.439160,34377.940540,11891.414170,12845.828470,-0.588686,-3.072583,-1.048065
19498,2011.0,b'O359859802',2.0,61.0,b'O359',160.1,67.2,26.217218,NaN,NaN,...,2177.872021,0.889561,1.083464,13564.637920,34958.357520,24210.578840,24950.322420,-0.315391,-1.170348,-0.949065
19499,2011.0,b'O359920901',1.0,65.0,b'O359',178.2,70.5,22.201060,NaN,NaN,...,2691.954983,1.028622,1.176135,9776.974162,36100.385010,15916.002950,16825.741910,-1.368321,-1.755417,-1.595887


In [4]:
# 9 빼기
final = final.loc[final['BS3_1'] != 9]
final

,year,ID,sex,age,psu,HE_ht,HE_wt,HE_BMI,HE_sbp_tr,HE_dbp_tr,...,DW_WBT_BMC,DW_SBT_BMD,DW_WBT_BMD,DW_Trk_FT,DW_Trk_MS,DW_SBT_FT,DW_WBT_FT,DX_F_Ts_A,DX_S_Ts_A,DX_FN_Ts_A
0,2008.0,b'A448440901',1.0,70.0,b'A448',155.2,58.0,24.079339,122.3,71.3,...,2113.500475,0.994057,1.119189,5350.559358,28165.253024,9588.845972,10554.355505,-0.893139,-0.537667,-1.857823
1,2008.0,b'A448440902',2.0,68.0,b'A448',165.6,63.4,23.118976,128.7,82.7,...,1616.637082,0.744743,0.851513,11248.239290,30770.948113,21544.371816,22368.168979,-1.420957,-1.534783,-2.336729
2,2008.0,b'A448680102',2.0,48.0,b'A448',152.3,59.3,25.565535,133.7,90.7,...,2274.388678,0.959672,1.237565,12183.372700,28925.836734,22269.339707,23114.327698,1.050435,1.171826,-0.459346
3,2008.0,b'A448680501',2.0,35.0,b'A448',162.5,53.4,20.222485,104.7,70.7,...,2292.951162,1.002821,1.182751,6573.697606,25808.635531,11724.950815,12571.863454,0.961565,0.354087,-0.260561
4,2008.0,b'A448740903',2.0,70.0,b'A448',155.7,63.9,26.358679,111.7,83.7,...,1455.752755,0.772691,0.882857,13489.507467,32492.389825,23471.432798,24379.078925,-1.764957,-3.223913,-2.883364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19496,2011.0,b'O359840901',2.0,60.0,b'O359',154.9,52.2,21.755430,NaN,NaN,...,1602.291088,0.785240,0.902542,9391.419342,25933.167250,16812.840990,17517.341590,-0.790261,-1.645652,-2.023738
19497,2011.0,b'O359859801',1.0,68.0,b'O359',165.2,68.2,24.989887,NaN,NaN,...,2062.206293,0.872690,0.953034,7434.439160,34377.940540,11891.414170,12845.828470,-0.588686,-3.072583,-1.048065
19498,2011.0,b'O359859802',2.0,61.0,b'O359',160.1,67.2,26.217218,NaN,NaN,...,2177.872021,0.889561,1.083464,13564.637920,34958.357520,24210.578840,24950.322420,-0.315391,-1.170348,-0.949065
19499,2011.0,b'O359920901',1.0,65.0,b'O359',178.2,70.5,22.201060,NaN,NaN,...,2691.954983,1.028622,1.176135,9776.974162,36100.385010,15916.002950,16825.741910,-1.368321,-1.755417,-1.595887


In [19]:
df = final.copy()

---------------------

In [20]:
df['HE_wt'] = df['HE_wt'] * 1000

In [21]:
df['LBM'] = 0.0
df['LBM'] = df['HE_wt'] - df['DW_WBT_FT']

In [22]:
for col in df.columns: 
    print(col, df[col].isna().sum())

year 0
ID 0
sex 0
age 0
psu 0
HE_ht 23
HE_wt 24
HE_BMI 31
HE_sbp_tr 2426
HE_dbp_tr 2426
HE_obe 187
HE_Upro 1564
HE_HbA1c 15632
HE_glu 851
HE_chol 809
HE_ast 809
HE_alt 809
HE_tb 494
HE_HP_tr 2652
DI3_lt 90
DI4_lt 178
HE_DM 1547
HE_wc 88
HE_HDL_st2 809
HE_LDL_drct 14789
HE_crea 812
D_1_1 90
L_BR1 4485
L_LN1 4485
L_DN1 4485
N_NA 2317
F_BEEF 4835
F_CHICK 4835
F_PORK 4835
BS3_1 98
BS3_2 98
BD1_11 98
BD2_1 98
BE3_21 98
BE3_22 98
BP1 98
DI3_dg 90
DI4_dg 178
DI1_dg 90
DE1_dg 90
DI2_dg 90
DI3_pt 90
DI4_pt 178
DI1_pt 90
DE1_pt 90
DI2_pt 90
HE_hepaB 809
BS6_2_2 98
BS6_3 98
BS2_1 98
pa_high 293
pa_mid 298
pa_walk 308
DJ1_dg 8567
DJ1_pt 8567
HW_MS 13770
DW_WBT_BMC 547
DW_SBT_BMD 547
DW_WBT_BMD 547
DW_Trk_FT 427
DW_Trk_MS 427
DW_SBT_FT 427
DW_WBT_FT 427
DX_F_Ts_A 356
DX_S_Ts_A 918
DX_FN_Ts_A 568
LBM 448


In [23]:
df.drop(['HE_HbA1c', 'HE_LDL_drct', 'HW_MS', 'DJ1_dg', 'DJ1_pt'], axis = 1, inplace = True)

In [ ]:
df.dropna(inplace = True)

In [31]:
df.reset_index(inplace = True, drop = True)

In [34]:
df

,year,ID,sex,age,psu,HE_ht,HE_wt,HE_BMI,HE_sbp_tr,HE_dbp_tr,...,DW_SBT_BMD,DW_WBT_BMD,DW_Trk_FT,DW_Trk_MS,DW_SBT_FT,DW_WBT_FT,DX_F_Ts_A,DX_S_Ts_A,DX_FN_Ts_A,LBM
0,2008.0,b'A448440902',2.0,68.0,b'A448',165.6,63400.0,23.118976,128.7,82.7,...,0.744743,0.851513,11248.239290,30770.948113,21544.371816,22368.168979,-1.420957,-1.534783,-2.336729,41031.831021
1,2008.0,b'A448680501',2.0,35.0,b'A448',162.5,53400.0,20.222485,104.7,70.7,...,1.002821,1.182751,6573.697606,25808.635531,11724.950815,12571.863454,0.961565,0.354087,-0.260561,40828.136546
2,2008.0,b'A448760901',1.0,75.0,b'A448',167.3,56900.0,20.329204,144.3,78.3,...,0.969161,1.117023,5546.540239,26378.934501,9978.956430,10973.370359,-0.850365,-1.970500,-1.843468,45926.629641
3,2008.0,b'A448760902',2.0,74.0,b'A448',147.8,47200.0,21.606933,144.7,94.7,...,0.809105,0.939216,7353.177333,23916.265929,12870.348994,13660.886036,-0.758261,-2.136870,-1.572617,33539.113964
4,2008.0,b'A448798401',1.0,44.0,b'A448',173.0,75000.0,25.059307,149.3,111.3,...,0.952682,1.045426,9143.372433,37888.893184,14533.752104,15516.731484,0.290949,-1.479083,-1.104194,59483.268516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11603,2010.0,b'P311679802',2.0,41.0,b'P311',159.9,58200.0,22.762820,111.9,75.9,...,0.859287,1.041685,9730.413776,26935.947910,20540.698900,21307.018780,0.757723,-0.473386,-0.708449,36892.981220
11604,2010.0,b'P311819601',1.0,36.0,b'P311',163.3,64200.0,24.074798,110.5,67.5,...,1.079230,1.241822,7181.059523,30594.145720,12144.028980,13080.741210,1.149071,-0.096513,0.503555,51119.258790
11605,2010.0,b'P311840701',1.0,63.0,b'P311',168.0,73800.0,26.147959,155.5,78.5,...,1.024554,1.142605,13518.139290,39434.636710,20923.938110,21939.460290,1.049084,-1.007729,0.475967,51860.539710
11606,2010.0,b'P311840702',2.0,61.0,b'P311',153.4,66200.0,28.132432,146.9,91.9,...,0.898079,1.061865,15622.174510,34184.632100,26416.873230,27378.956380,-0.037247,-0.849528,-1.098833,38821.043620


In [35]:
df.to_csv('LBMstepwise_1013.csv',encoding = 'euc-kr', index = False)

In [38]:
for col in df.columns: 
    print(df[col].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [33]:
## Stepwise selection

variables = df.columns.tolist() ## 설명 변수 리스트
variables.remove('year')
variables.remove('ID')
variables.remove('psu')
print(variables)
 
y = df['LBM'] ## 반응 변수
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05
sl_remove = 0.05
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = df[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = df[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(df[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

['sex', 'age', 'HE_ht', 'HE_wt', 'HE_BMI', 'HE_sbp_tr', 'HE_dbp_tr', 'HE_obe', 'HE_Upro', 'HE_glu', 'HE_chol', 'HE_ast', 'HE_alt', 'HE_tb', 'HE_HP_tr', 'DI3_lt', 'DI4_lt', 'HE_DM', 'HE_wc', 'HE_HDL_st2', 'HE_crea', 'D_1_1', 'L_BR1', 'L_LN1', 'L_DN1', 'N_NA', 'F_BEEF', 'F_CHICK', 'F_PORK', 'BS3_1', 'BS3_2', 'BD1_11', 'BD2_1', 'BE3_21', 'BE3_22', 'BP1', 'DI3_dg', 'DI4_dg', 'DI1_dg', 'DE1_dg', 'DI2_dg', 'DI3_pt', 'DI4_pt', 'DI1_pt', 'DE1_pt', 'DI2_pt', 'HE_hepaB', 'BS6_2_2', 'BS6_3', 'BS2_1', 'pa_high', 'pa_mid', 'pa_walk', 'DW_WBT_BMC', 'DW_SBT_BMD', 'DW_WBT_BMD', 'DW_Trk_FT', 'DW_Trk_MS', 'DW_SBT_FT', 'DW_WBT_FT', 'DX_F_Ts_A', 'DX_S_Ts_A', 'DX_FN_Ts_A', 'LBM']


<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

...

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 변수의 p-value
<ipython-input-33-87dd9241db5a>:20: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pval = pd.Series(index=remainder) ## 

KeyboardInterrupt: 

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

------------------------------------------------------

In [6]:
# 필요한 변수만 추출

feature_col = ['DW_WBT_FT', 'DW_WBT_BMC', 'HE_wt', 'HE_wc', 'BS3_1', 'HE_ht', 'sex', 'HE_chol', 'age', 'HE_sbp_tr']
df = df[feature_col]

In [7]:
df

,DW_WBT_FT,DW_WBT_BMC,HE_wt,HE_wc,BS3_1,HE_ht,sex,HE_chol,age,HE_sbp_tr
0,10554.355505,2113.500475,58.0,75.2,1.0,155.2,1.0,144.0,70.0,122.3
1,22368.168979,1616.637082,63.4,87.5,8.0,165.6,2.0,248.0,68.0,128.7
2,23114.327698,2274.388678,59.3,81.0,8.0,152.3,2.0,199.0,48.0,133.7
3,12571.863454,2292.951162,53.4,65.1,8.0,162.5,2.0,200.0,35.0,104.7
4,24379.078925,1455.752755,63.9,86.5,8.0,155.7,2.0,161.0,70.0,111.7
...,...,...,...,...,...,...,...,...,...,...
19496,17517.341590,1602.291088,52.2,70.3,8.0,154.9,2.0,189.0,60.0,NaN
19497,12845.828470,2062.206293,68.2,85.5,2.0,165.2,1.0,199.0,68.0,NaN
19498,24950.322420,2177.872021,67.2,90.0,8.0,160.1,2.0,218.0,61.0,NaN
19499,16825.741910,2691.954983,70.5,88.0,2.0,178.2,1.0,106.0,65.0,NaN


In [95]:
# df.drop(['DW_WBT_FT'], axis = 1, inplace = True)
# df

,DW_WBT_BMC,HE_wt,HE_wc,BS3_1,HE_ht,sex,HE_chol,age,HE_sbp_tr,LBM
0,2113.500475,58000.0,75.2,1.0,155.2,1.0,144.0,70.0,122.3,47445.644495
1,1616.637082,63400.0,87.5,8.0,165.6,2.0,248.0,68.0,128.7,41031.831021
2,2274.388678,59300.0,81.0,8.0,152.3,2.0,199.0,48.0,133.7,36185.672302
3,2292.951162,53400.0,65.1,8.0,162.5,2.0,200.0,35.0,104.7,40828.136546
4,1455.752755,63900.0,86.5,8.0,155.7,2.0,161.0,70.0,111.7,39520.921075
...,...,...,...,...,...,...,...,...,...,...
19496,1602.291088,52200.0,70.3,8.0,154.9,2.0,189.0,60.0,NaN,34682.658410
19497,2062.206293,68200.0,85.5,2.0,165.2,1.0,199.0,68.0,NaN,55354.171530
19498,2177.872021,67200.0,90.0,8.0,160.1,2.0,218.0,61.0,NaN,42249.677580
19499,2691.954983,70500.0,88.0,2.0,178.2,1.0,106.0,65.0,NaN,53674.258090


In [10]:
df.dropna(inplace = True)
df.reset_index(inplace = True, drop = True)
df

,DW_WBT_FT,DW_WBT_BMC,HE_wt,HE_wc,BS3_1,HE_ht,sex,HE_chol,age,HE_sbp_tr,LBM
0,10554.355505,2113.500475,58000.0,75.2,1.0,155.2,1.0,144.0,70.0,122.3,47445.644495
1,22368.168979,1616.637082,63400.0,87.5,8.0,165.6,2.0,248.0,68.0,128.7,41031.831021
2,23114.327698,2274.388678,59300.0,81.0,8.0,152.3,2.0,199.0,48.0,133.7,36185.672302
3,12571.863454,2292.951162,53400.0,65.1,8.0,162.5,2.0,200.0,35.0,104.7,40828.136546
4,24379.078925,1455.752755,63900.0,86.5,8.0,155.7,2.0,161.0,70.0,111.7,39520.921075
...,...,...,...,...,...,...,...,...,...,...,...
15766,23666.906780,2142.047296,66800.0,84.6,1.0,163.3,1.0,160.0,47.0,101.5,43133.093220
15767,11928.305590,1517.783443,43600.0,64.8,8.0,148.8,2.0,172.0,43.0,97.9,31671.694410
15768,33872.773520,2028.471021,75000.0,98.7,8.0,157.9,2.0,187.0,76.0,138.9,41127.226480
15769,21495.671590,3028.905384,73000.0,92.0,2.0,169.5,1.0,138.0,47.0,132.5,51504.328410


In [11]:
df.isna().sum()

DW_WBT_FT     0
DW_WBT_BMC    0
HE_wt         0
HE_wc         0
BS3_1         0
HE_ht         0
sex           0
HE_chol       0
age           0
HE_sbp_tr     0
LBM           0
dtype: int64

In [12]:
dummy_cols = ['BS3_1', 'sex']
df = pd.get_dummies(df, columns = dummy_cols)
df

,DW_WBT_FT,DW_WBT_BMC,HE_wt,HE_wc,HE_ht,HE_chol,age,HE_sbp_tr,LBM,BS3_1_1.0,BS3_1_2.0,BS3_1_8.0,sex_1.0,sex_2.0
0,10554.355505,2113.500475,58000.0,75.2,155.2,144.0,70.0,122.3,47445.644495,1,0,0,1,0
1,22368.168979,1616.637082,63400.0,87.5,165.6,248.0,68.0,128.7,41031.831021,0,0,1,0,1
2,23114.327698,2274.388678,59300.0,81.0,152.3,199.0,48.0,133.7,36185.672302,0,0,1,0,1
3,12571.863454,2292.951162,53400.0,65.1,162.5,200.0,35.0,104.7,40828.136546,0,0,1,0,1
4,24379.078925,1455.752755,63900.0,86.5,155.7,161.0,70.0,111.7,39520.921075,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15766,23666.906780,2142.047296,66800.0,84.6,163.3,160.0,47.0,101.5,43133.093220,1,0,0,1,0
15767,11928.305590,1517.783443,43600.0,64.8,148.8,172.0,43.0,97.9,31671.694410,0,0,1,0,1
15768,33872.773520,2028.471021,75000.0,98.7,157.9,187.0,76.0,138.9,41127.226480,0,0,1,0,1
15769,21495.671590,3028.905384,73000.0,92.0,169.5,138.0,47.0,132.5,51504.328410,0,1,0,1,0


In [13]:
# bias를 위한 상수항 추가

df = sm.add_constant(df, has_constant = "add")
df

,const,DW_WBT_FT,DW_WBT_BMC,HE_wt,HE_wc,HE_ht,HE_chol,age,HE_sbp_tr,LBM,BS3_1_1.0,BS3_1_2.0,BS3_1_8.0,sex_1.0,sex_2.0
0,1.0,10554.355505,2113.500475,58000.0,75.2,155.2,144.0,70.0,122.3,47445.644495,1,0,0,1,0
1,1.0,22368.168979,1616.637082,63400.0,87.5,165.6,248.0,68.0,128.7,41031.831021,0,0,1,0,1
2,1.0,23114.327698,2274.388678,59300.0,81.0,152.3,199.0,48.0,133.7,36185.672302,0,0,1,0,1
3,1.0,12571.863454,2292.951162,53400.0,65.1,162.5,200.0,35.0,104.7,40828.136546,0,0,1,0,1
4,1.0,24379.078925,1455.752755,63900.0,86.5,155.7,161.0,70.0,111.7,39520.921075,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15766,1.0,23666.906780,2142.047296,66800.0,84.6,163.3,160.0,47.0,101.5,43133.093220,1,0,0,1,0
15767,1.0,11928.305590,1517.783443,43600.0,64.8,148.8,172.0,43.0,97.9,31671.694410,0,0,1,0,1
15768,1.0,33872.773520,2028.471021,75000.0,98.7,157.9,187.0,76.0,138.9,41127.226480,0,0,1,0,1
15769,1.0,21495.671590,3028.905384,73000.0,92.0,169.5,138.0,47.0,132.5,51504.328410,0,1,0,1,0


In [107]:
selected_variables = ['const', 'HE_wt', 'HE_wc', 'BS3_1_1.0', 'BS3_1_2.0', 'HE_ht', 'HE_chol', 'sex_1.0', 'age', 'HE_sbp_tr']

In [108]:
# train test split - 종속변수가 총골량일때 
# using selected variables

feature_columns = df.columns.difference(["DW_WBT_BMC", "LBM"]) # Target column 빼기
feature_columns = selected_variables

X_fat = df[feature_columns]
y_fat = df.DW_WBT_BMC

train_x_fat, test_x_fat, train_y_fat, test_y_fat = train_test_split(X_fat,y_fat, train_size = 0.7, test_size = 0.3)

print(train_x_fat.shape, test_x_fat.shape, train_y_fat.shape, test_y_fat.shape)

(11039, 10) (4732, 10) (11039,) (4732,)


In [109]:
# 총골량 OLS

model = sm.OLS(train_y_fat,train_x_fat)
fitted_model = model.fit()

fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             DW_WBT_BMC   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     2807.
Date:                Thu, 07 Oct 2021   Prob (F-statistic):               0.00
Time:                        11:33:19   Log-Likelihood:                -77212.
No. Observations:               11039   AIC:                         1.544e+05
Df Residuals:                   11029   BIC:                         1.545e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -875.9920     95.426     -9.180      0.000   -1063.043    -688.941
HE_wt          0.0220      0.001     36.502      0.000       0.021       0.023
HE_wc         -7.6246      0.583    -13.073      0.000      -8.768      -6.481
BS3_1_1.0     -6.0251      7.294     -0.826      0.409     -20.323       8.273
BS3_1_2.0      8.8753     11.023      0.805      0.421     -12.732      30.483
HE_ht         15.9602      0.561     28.466      0.000      14.861      17.059
HE_chol       -0.5249      0.074     -7.124      0.000      -0.669      -0.381
sex_1.0      173.4489      9.218     18.816      0.000     155.380     191.518
age           -2.3491      0.230    -10.231      0.000      -2.799      -1.899
HE_sbp_tr     -0.9829      0.172     -5.712      0.000      -1.320      -0.646
==============================================================================
Omnibus:                      178.973   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              189.162
Skew:                           0.304   Prob(JB):                     8.39e-42
Kurtosis:                       3.207   Cond. No.                     2.41e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.41e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [110]:
# train test split - 종속변수가 LBM일때 
# using selected variables

feature_columns = df.columns.difference(["DW_WBT_BMC", "LBM"]) # Target column 빼기
feature_columns = selected_variables

X_fat = df[feature_columns]
y_fat = df.LBM

train_x_fat, test_x_fat, train_y_fat, test_y_fat = train_test_split(X_fat,y_fat, train_size = 0.7, test_size = 0.3)

print(train_x_fat.shape, test_x_fat.shape, train_y_fat.shape, test_y_fat.shape)

(11039, 10) (4732, 10) (11039,) (4732,)


In [111]:
# 총골량 OLS

model = sm.OLS(train_y_fat,train_x_fat)
fitted_model = model.fit()

fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    LBM   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.932
Method:                 Least Squares   F-statistic:                 1.675e+04
Date:                Thu, 07 Oct 2021   Prob (F-statistic):               0.00
Time:                        11:33:30   Log-Likelihood:            -1.0224e+05
No. Observations:               11039   AIC:                         2.045e+05
Df Residuals:                   11029   BIC:                         2.046e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7958.6414    918.053     -8.669      0.000   -9758.189   -6159.094
HE_wt          0.5946      0.006    100.788      0.000       0.583       0.606
HE_wc       -131.7576      5.775    -22.815      0.000    -143.078    -120.438
BS3_1_1.0    418.1397     70.313      5.947      0.000     280.315     555.965
BS3_1_2.0  -1009.8637    106.544     -9.478      0.000   -1218.708    -801.019
HE_ht        142.9204      5.394     26.496      0.000     132.347     153.494
HE_chol       -5.8732      0.705     -8.335      0.000      -7.255      -4.492
sex_1.0     7137.7999     88.285     80.850      0.000    6964.746    7310.854
age           12.6480      2.213      5.716      0.000       8.311      16.985
HE_sbp_tr      5.1267      1.673      3.063      0.002       1.846       8.407
==============================================================================
Omnibus:                      109.547   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              164.704
Skew:                          -0.098   Prob(JB):                     1.72e-36
Kurtosis:                       3.565   Cond. No.                     2.40e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
df.to_csv('testdf1007_LBM+ASM_final.csv', encoding = 'euc-kr', index = False)